In [1]:
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np
from numba import jit
import pstats
import heapq
import time
import cProfile
import pandas as pd

In [2]:
puzzle_info_path = 'puzzle_info.csv'
puzzles_path = 'puzzles.csv'
sample_submission_path = 'sample_submission.csv' # change name across board
puzzle_info_df = pd.read_csv(puzzle_info_path)
puzzles_df = pd.read_csv(puzzles_path)
sample_submission_df = pd.read_csv(sample_submission_path)

In [3]:
# Parsing the initial_state and solution_state columns
# Converting the semicolon-separated string values into lists of colors
puzzles_df['parsed_initial_state'] = puzzles_df['initial_state'].apply(lambda x: x.split(';'))
seen = {}

for i in range(len(puzzles_df['parsed_initial_state'])):
    for j in range(len(puzzles_df['parsed_initial_state'][i])):
        if puzzles_df['parsed_initial_state'][i][j] not in seen:
            seen[puzzles_df['parsed_initial_state'][i][j]] = len(seen)
        puzzles_df['parsed_initial_state'][i][j] = seen[puzzles_df['parsed_initial_state'][i][j]]

puzzles_df['parsed_solution_state'] = puzzles_df['solution_state'].apply(lambda x: x.split(';'))

for i in range(len(puzzles_df['parsed_solution_state'])):
    for j in range(len(puzzles_df['parsed_solution_state'][i])):
        puzzles_df['parsed_solution_state'][i][j] = seen[puzzles_df['parsed_solution_state'][i][j]]

In [4]:
import json

# Converting the string representation of allowed_moves to dictionary
puzzle_info_df['allowed_moves'] = puzzle_info_df['allowed_moves'].apply(lambda x: json.loads(x.replace("'", '"')))

# Selecting an example puzzle type and displaying its allowed moves
example_puzzle_type = puzzle_info_df['puzzle_type'].iloc[0]
example_allowed_moves = puzzle_info_df[puzzle_info_df['puzzle_type'] == example_puzzle_type]['allowed_moves'].iloc[0]

In [5]:
def getInversePerm(arr):
    # gets the inverse move for a certain move
    res = [0 for i in range(len(arr))]
    for i in range(len(arr)):
        res[arr[i]] = i
    return res
    
# type : (np.array(move_perm_i), np.array(name_i))
puz_info = {}

# type : {move : perm}
move_to_perm = {}

for i in range(len(puzzle_info_df)):
    puz_info[puzzle_info_df['puzzle_type'][i]] = [[], []]
    move_to_perm[puzzle_info_df['puzzle_type'][i]] = {}
    
    for j in puzzle_info_df['allowed_moves'][i].keys():
        puz_info[puzzle_info_df['puzzle_type'][i]][1].append(j)
        puz_info[puzzle_info_df['puzzle_type'][i]][0].append(np.array(puzzle_info_df['allowed_moves'][i][j]))

        puz_info[puzzle_info_df['puzzle_type'][i]][1].append(str('-' + j)) # might be the opposite
        puz_info[puzzle_info_df['puzzle_type'][i]][0].append(np.array(getInversePerm(puzzle_info_df['allowed_moves'][i][j])))

        move_to_perm[puzzle_info_df['puzzle_type'][i]][str('-' + j)] = np.array(getInversePerm(puzzle_info_df['allowed_moves'][i][j]))
        move_to_perm[puzzle_info_df['puzzle_type'][i]][j] = np.array(puzzle_info_df['allowed_moves'][i][j])

# move_to_perm['cube_2/2/2']

In [24]:
@jit(nopython=True, parallel = True, fastmath = True)
def hash_perm(perm):
    base = 9973
    modb = 1000000007
    modc = 1000000009

    B, C = 0, 0
    for i in perm:
        B = (B * base) % modb + i
        C = (C * base) % modc + i

    return (B, C)

def dist(a, b):
    return np.count_nonzero(a != b)

In [25]:
# 1e6 ~ 2 seconds
mx_mem = int(1e7)
mem_idx = 0

last_state = np.zeros(mx_mem, dtype=int)
last_move = np.zeros(mx_mem, dtype=int)

print(last_state)

[0 0 0 ... 0 0 0]


In [76]:
def chunk_search(idx, initial_state = None, chunk_time = 1):
    global mem_idx

    if initial_state is None:
        initial_state = np.array(puzzles_df['parsed_initial_state'][idx])
    
    goal_state = np.array(puzzles_df['parsed_solution_state'][idx])
    max_dist = puzzles_df['num_wildcards'][idx]
    move_perm = np.array(puz_info[puzzles_df['puzzle_type'][idx]][0])

    n = len(initial_state)
    m = len(move_perm)

    avg_dist = np.array([n + 1 for i in range(500 * n)], dtype = np.float32)
    n_dist = np.array([0 for i in range(500 * n)])

    def upd_prune(nxt_p, nxt_dist, avg_dist, n_dist):
        if n_dist[nxt_p] == 0:
            avg_dist[nxt_p] = nxt_dist
            n_dist[nxt_p] = 1
            return True

        if n_dist[nxt_p] > 500: # pruning here
            if nxt_dist > avg_dist[nxt_p]:
                return False
        
        avg_dist[nxt_p] *= n_dist[nxt_p] / (n_dist[nxt_p] + 1)
        avg_dist[nxt_p] += nxt_dist / (n_dist[nxt_p] + 1)
        n_dist[nxt_p] += 1
        return True
    
    pq = []
    heapq.heappush(pq, (0, mem_idx, initial_state, 0))  # (priority, mem_idx, state, path_length)
    mem_idx += 1

    best_branch = (dist(initial_state, goal_state), 0, initial_state, 0) # (score, mem_idx, state, path_length)

    considered = set()
    considered.add(hash_perm(initial_state))

    start_time = time.time()
    
    while mem_idx + m < mx_mem:
        if time.time() - start_time > chunk_time:
            break
        
        cur_p_plus_dist, cur_idx, cur_state, cur_p = heapq.heappop(pq) 

        for i in range(m):
            nxt_state = cur_state[move_perm[i]]
            nxt_hash = hash_perm(nxt_state)
            nxt_p = 1 + cur_p

            nxt_dist = dist(nxt_state, goal_state)
            
            ok = upd_prune(nxt_p, nxt_dist,avg_dist,n_dist)
            if not ok:
                continue

            if nxt_hash in considered:
                continue
            considered.add(nxt_hash)

            if (nxt_dist == best_branch[0] and nxt_p < best_branch[3]) or nxt_dist < best_branch[0]:
                # print(nxt_dist, nxt_p)
                best_branch = (nxt_dist, mem_idx, nxt_state, nxt_p)
            
            last_state[mem_idx] = cur_idx
            last_move[mem_idx] = i

            if nxt_dist <= max_dist:
                # print("Number of pruned branches", pruned_branch)
                # print("Chunk length", cur_p)
                # print("Chunk Dist", nxt_dist)
                
                return best_branch
            
            priority = nxt_p + nxt_dist
            heapq.heappush(pq, (priority, mem_idx, nxt_state, 1+cur_p))
            mem_idx += 1
    return best_branch

mem_idx = 0
print(chunk_search(30))
# cProfile.run('chunk_search(30)')

(13, 37949, array([5, 4, 2, 2, 2, 2, 2, 2, 2, 1, 3, 3, 3, 3, 4, 3, 3, 3, 4, 0, 4, 2,
       4, 4, 4, 4, 3, 0, 0, 0, 5, 0, 0, 1, 0, 0, 4, 1, 0, 1, 1, 1, 1, 1,
       1, 5, 5, 5, 5, 5, 3, 5, 5, 2]), 20)


In [78]:
def dig_through_memory(idx):
    res = []
    while idx != 0:
        res.append(last_move[idx])
        idx = last_state[idx]

    res = list(reversed(res))
    return res

def solve(idx, allotted_time = 10):
    global mem_idx
    
    start_time = time.time()
    move_name = np.array(puz_info[puzzles_df['puzzle_type'][idx]][1])
    max_dist = puzzles_df['num_wildcards'][idx]

    found = 0
    res = []
    cur = np.array(puzzles_df['parsed_initial_state'][idx])
    # chunk_time = 1
    chunk_time = allotted_time + 1
    
    while (time.time() - start_time < allotted_time and not found):
        # print("Start chunk")
        mem_idx = 0
        nxt_cur = chunk_search(idx, cur, chunk_time = chunk_time) 
        # larger chunk_time -> better solution, longer time
        # more breath on search

        if nxt_cur[0] <= max_dist:
            found = 1

        cur_moves = dig_through_memory(nxt_cur[1])
        res = res + cur_moves
        
        cur = nxt_cur[2]
        chunk_time *= 2
        # print("End chunk")
    
    if not found:
        return None
        
    for i in range(len(res)):
        res[i] = move_name[res[i]]
    return res

A = solve(5)
A

['r0',
 'f0',
 'd1',
 '-f0',
 'r0',
 '-f0',
 '-r0',
 'f0',
 'f0',
 '-r0',
 '-f0',
 'r0',
 'd1',
 'r0',
 '-d1',
 '-f0',
 '-d1',
 'f0',
 'd1',
 'r0',
 'd1',
 '-r0',
 '-d1',
 'r0',
 'd1',
 '-r0',
 '-d1',
 'r0',
 '-d1',
 '-r0',
 '-f0',
 'r1',
 'f0',
 '-r1',
 '-d0',
 '-r1',
 'd0',
 'r1',
 'f0',
 '-r1',
 '-f0',
 'r1',
 '-d0',
 '-r1',
 'f0',
 'd0',
 '-r0',
 '-d1',
 'f0',
 'd1',
 '-f0',
 '-r1',
 'd0',
 'r1',
 '-d0',
 'r1',
 '-f0',
 'd0',
 'f0',
 '-r1',
 'f0',
 'r1',
 '-f0',
 '-f0',
 'r0',
 'f0',
 '-r0',
 '-r0',
 'f0',
 'r0',
 'd1',
 'r0',
 '-d1',
 'f0',
 '-r0',
 '-f0']

# Step 6: Build submission format function

In [ ]:
def format_solution_for_submission(puzzle_id, solution_moves):
    """
    Format the solution to a puzzle for submission.

    :param puzzle_id: The unique identifier of the puzzle.
    :param solution_moves: List of tuples representing the solution moves.
    :return: Formatted string suitable for submission.
    """
    formatted_moves = []
    for move in solution_moves:
        formatted_moves.append(move)

    # Joining the moves into a single string separated by periods
    return {'id': puzzle_id, 'moves': '.'.join(formatted_moves)}


# Step 7: Define solve function

In [82]:
from tqdm import tqdm

all_ok = ['cube_3/3/3', 'wreath_21/21', 'wreath_33/33']
res_df = []

for i in tqdm(range(len(puzzles_df))):
    res = None
    print(i)
    type = puzzles_df['puzzle_type'][i]
    if type in all_ok:
        res = solve(i, 90)
    
    if res is None:
        res = format_solution_for_submission(i, sample_submission_df['moves'][i].split('.'))
    else:
        res = format_solution_for_submission(i, res)
    
    res_df.append(res)

  0%|                                                   | 0/398 [00:00<?, ?it/s]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30


  8%|███▎                                      | 31/398 [01:01<12:09,  1.99s/it]

31


  8%|███▍                                      | 32/398 [01:04<12:16,  2.01s/it]

32


  8%|███▍                                      | 32/398 [02:32<29:05,  4.77s/it]


KeyboardInterrupt: 

In [ ]:
last_submission_df = pd.read_csv('submission.csv')
last_submission_df

In [ ]:
res_df = pd.DataFrame(res_df)
res_df

In [ ]:
# Define the file path for the output CSV file
output_csv_path = 'submission.csv'

# Save the output DataFrame to a CSV file
res_df.to_csv(output_csv_path, index=False)

# Return the path of the saved file
output_csv_path